In [89]:

import pandas as pd 

In [90]:
users = pd.read_csv('ml-latest-small/ratings.csv', sep=',',
 encoding='latin-1')



In [91]:
df=users.groupby(['userId']).count()
n_users,k=df.shape
print(n_users)

610


In [92]:
ratings_base = pd.read_csv('ml-latest-small/ratings.csv', sep=',', encoding='latin-1')

ratings_base=ratings_base.loc[ratings_base.movieId<9743]
ratings_base.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [93]:
rate_train = ratings_base.to_numpy()
print(rate_train.shape)


(77939, 4)


In [94]:
movies_df = pd.read_csv('ml-latest-small/movies.csv',sep=',',encoding='latin-1')
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

C:\Users\MyPC\AppData\Local\Temp/ipykernel_12948/3141824109.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [96]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [97]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df=moviesWithGenres_df.iloc[:,4:].astype(int)
moviesWithGenres_df.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [98]:
items=moviesWithGenres_df
X0=items.to_numpy()
X_train_counts = X0[:, -20:-2]
print(X_train_counts.shape)

(9742, 18)


In [99]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

print(tfidf.shape)

(9742, 18)


In [100]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    return (item_ids, scores)
    """
    y = rate_matrix[:,0] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1 
    # but id in python starts from 0
    ids = np.where(y == user_id +1 )[0] 
    item_ids = rate_matrix[ids, 1] -1 # index starts from 0 
    scores = rate_train[ids, 2]
    return (item_ids, scores)

In [56]:
 ids = np.where(y == 1 +1)[0] 

In [102]:
ids, scores = get_items_rated_by_user(rate_train,1)
print(ids)

[ 317.  332. 1703. 3577. 6873. 8797.]


In [103]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = tfidf.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))


for n in range(n_users):    
    ids, scores = get_items_rated_by_user(rate_train, n)
    ids=ids.astype(int)
    clf = Ridge(alpha=0.01, fit_intercept  = True)
   
    Xhat = tfidf[ids, :]
    
    clf.fit(Xhat, scores) 
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

In [104]:
Yhat = tfidf.dot(W) + b

In [114]:
n = 100
ids, scores = get_items_rated_by_user(rate_train, 4)
ids=ids.astype(int)
Yhat[n, ids]
print ('Rated movies ids:', ids )
print ('True ratings:', scores)
print ('Predicted ratings:', Yhat[ids, n])

Rated movies ids: [  0  20  33  35  38  49  57 109 149 152 231 246 252 260 264 265 289 295
 299 315 317 343 348 356 363 366 379 409 456 473 474 514 526 530 533 587
 588 589 591 593 594 595 596 607]
True ratings: [4. 4. 4. 4. 3. 4. 5. 4. 3. 3. 4. 5. 3. 4. 3. 1. 5. 5. 3. 2. 3. 3. 3. 2.
 3. 4. 2. 3. 4. 4. 5. 3. 5. 4. 3. 4. 3. 5. 3. 5. 5. 5. 3. 3.]
Predicted ratings: [3.41640185 3.59051013 4.02666224 2.80224709 4.13028336 5.06636018
 2.80224709 4.4152318  0.46911027 2.80224709 3.86165329 4.00984623
 3.50802972 3.86165329 3.68072118 3.38049731 3.86165329 3.86165329
 3.86165329 2.80224709 3.07112664 3.86165329 4.02666224 3.86165329
 3.38049731 3.23593609 4.38400108 3.38049731 3.63703124 3.23593609
 4.27593781 2.80224709 3.68072118 3.38049731 4.88711786 1.76607059
 3.38049731 3.86165329 3.63703124 0.46911027 2.96898619 4.55284106
 3.02060971 3.77456008]


In [119]:
from math import sqrt
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        ids=ids.astype(int)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return sqrt(se/cnt)

print ('RMSE for training:', evaluate(Yhat, rate_train, W, b))


RMSE for training: 0.8858674049284931
